In [2]:
%pip install mitiq
%pip install qiskit-aer
%pip install qiskit-ibm-provider

26.76s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.
33.24s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.
39.11s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.


In [3]:
import qiskit
from qiskit_aer import Aer

from mitiq import zne, pec
from mitiq.interface.mitiq_qiskit.qiskit_utils import initialized_depolarizing_noise
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise

In [4]:
qreg, creg = qiskit.QuantumRegister(1), qiskit.ClassicalRegister(1)
circuit = qiskit.QuantumCircuit(qreg, creg)
for _ in range(10):
    circuit.x(qreg)
circuit.measure(qreg, creg)
print(circuit)

      ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┐
  q1: ┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤M├
      └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└╥┘
c0: 1/═══════════════════════════════════════════════════╩═
                                                         0 


In [5]:
backend = Aer.get_backend("qasm_simulator")
noise_level=0.01
number_of_shots = 8192

def executor(circuit: qiskit.QuantumCircuit, noise_level = noise_level) -> float:
    noise_model = initialized_depolarizing_noise(noise_level=noise_level)
    job = qiskit.execute(
        experiments=circuit,
        backend=backend,
        noise_model=noise_model,
        basis_gates=noise_model.basis_gates,
        optimization_level=0,# Important to preserve folded gates.
        shots=number_of_shots,
    )

    # Convert from raw measurement counts to the expectation value
    counts = job.result().get_counts()
    if counts.get("0") is None:
        expectation_value = 0.
    else:
        expectation_value = counts.get("0") / number_of_shots
    return expectation_value

unmitigated = executor(circuit)

In [9]:
#zne example
mitigated = zne.execute_with_zne(circuit, executor)
# what happens under the hood of calling execute_with_zne method.
# scale_factors = [1., 1.5, 2., 2.5, 3.]
# folded_circuits = [
#         zne.scaling.fold_gates_at_random(circuit, scale)
#         for scale in scale_factors
# ]

# # Check that the circuit depth is (approximately) scaled as expected
# for j, c in enumerate(folded_circuits):
#     print(f"Number of gates of folded circuit {j} scaled by: {len(c) / len(circuit):.3f}")

print(f"Unmitigated result {unmitigated:.3f}")
print(f"Mitigated result {mitigated:.3f}")

Unmitigated result 0.955
Mitigated result 1.006


In [7]:
#pec example
#warnings due to the fact that operationrepresentation can't be prepared for circuits that are measured. Should be discussed. 
reps = represent_operations_in_circuit_with_local_depolarizing_noise(circuit, noise_level)
print(f"{len(reps)} OperationRepresentation objects produced, assuming {100 * noise_level}% depolarizing noise.")
pec_value = pec.execute_with_pec(circuit, executor, representations=reps)

print(f"Unmitigated result {unmitigated:.3f}")
print(f"Mitigated result {pec_value:.3f}")

1 OperationRepresentation objects produced, assuming 1.0% depolarizing noise.


/Users/markiiantsymbalista/Desktop/runtime-prototype/runtime/.venv/lib/python3.11/site-packages/mitiq/pec/sampling.py:75: UserWarning: No representation found for 

q1_0: ───M('c0_0')───.
  warnings.warn(
/Users/markiiantsymbalista/Desktop/runtime-prototype/runtime/.venv/lib/python3.11/site-packages/mitiq/pec/sampling.py:75: UserWarning: No representation found for 

q1_0: ───M('c0_0')───.
  warnings.warn(


Unmitigated result 0.955
Mitigated result 1.018


In [8]:
# #CDR
# import numpy as np
# import warnings
# warnings.simplefilter("ignore", np.ComplexWarning)

# import cirq
# from mitiq import cdr, Observable, PauliString

# # Define two qubits
# a, b = 0, 1  
# # Create a quantum circuit
# cdr_circuit = qiskit.QuantumCircuit(2)
# # Apply Hadamard gates (Clifford)
# cdr_circuit.h(a)
# cdr_circuit.h(b)
# # Apply rz gates
# cdr_circuit.rz(1.75, a)
# cdr_circuit.rz(2.31, b)
# # Apply CNOT gate (Clifford)
# cdr_circuit.cx(a, b)
# # Apply rz gates
# cdr_circuit.rz(-1.17, b)
# cdr_circuit.rz(3.23, a)
# # Apply rx gates (Clifford)
# cdr_circuit.rx(np.pi / 2, a)
# cdr_circuit.rx(np.pi / 2, b)
# # Display the circuit
# print(cdr_circuit)

# from mitiq.interface.mitiq_cirq import compute_density_matrix

# compute_density_matrix(circuit).round(3)